# Clean a single P3 TDR Scan

### Import Statements

In [3]:
import math
import os, sys
import numpy as np
#Workarounds for new numpy install - Uncomment Below if needed
#np.object = object
#np.int = int
#np.float = float
#np.bool = bool
#np.typeDict = np.sctypeDict
from scipy import ndimage
import netCDF4 as nc4
import re
import h5py
import glob

import sys, argparse
import pandas as pd
import pickle

# from keras.models import Sequential
# from keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU
# from keras.models import model_from_yaml
# from keras.callbacks import EarlyStopping, ModelCheckpoint
# from keras.callbacks import TensorBoard

# from sklearn.model_selection import train_test_split
# from sklearn import svm
# from sklearn import metrics
# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import classification_report

from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

import matplotlib as mpl
import matplotlib.pyplot as plt
from tensorflow import keras

from utils import *


##Isaac module additions
from os import listdir 

In [10]:
fill_val = int(-32768)

def write_to_cfrad(path, variable, var_name):
    cfrad = nc4.Dataset(path, 'r+')
    try:
        currvar = cfrad.createVariable(var_name, 'i2', ('time','range',), fill_value=fill_val)
        currvar[:,:] = variable
    except RuntimeError:
        cfrad[var_name] = variable
    finally: 
        print("BRUH")
        cfrad.close()



In [4]:
###Function for calculating PGG and AHT 
def get_PGG(cfrad):

    max_time = cfrad.dimensions['time'].size
    max_range = cfrad.dimensions['range'].size

    rnge = cfrad.variables['range']
    plane_alts = cfrad.variables['altitude']
    ray_angles = cfrad.variables['elevation']
    ranges = cfrad.variables['range']
    azimuths = cfrad.variables['azimuth']

    

    heights = np.empty( (max_time, max_range) )
    pgg = np.empty( (max_time, max_range) )

    for time_idx in range(max_time):
        for range_idx in range(max_range):
            tangle = float(ray_angles[time_idx].data)
            trange = float(ranges[range_idx].data)
            talt = float(plane_alts[time_idx].data)
            taz = float(azimuths[time_idx].data)
            
            heights[time_idx, range_idx] = computeHtKmAirborn(tangle, trange, talt)

            pgg[time_idx, range_idx] = probGroundGates(tangle, trange, talt, taz, max_range)
            

    return(heights, pgg)


In [5]:
cfrad = nc4.Dataset(cfrad_loc)
SVEL = compute_std(cfrad["VEL"][:,:], -32800)
IVV = compute_isolation(cfrad["VEL"][:,:], -32800)
cfrad.close()

NameError: name 'cfrad_loc' is not defined

In [69]:
write_to_cfrad(cfrad_loc, heights, "AHT")
write_to_cfrad(cfrad_loc, pgg, "PGG")
write_to_cfrad(cfrad_loc, SVEL, "SVV")
write_to_cfrad(cfrad_loc, IVV, "ISO(VEL)")

BRUH
BRUH
BRUH
BRUH


In [18]:
os.getcwd()

'/Users/ischluesche/Documents/Grad_School/Research/RadarQC/ALEX_VERIFICATION/Walkthrough_notebook'

In [28]:
cfrad = nc4.Dataset("./cfrad/cfrad.20220918_124128.197_to_20220918_124132.176_N42RF-TS_AIR.nc")

In [12]:
def load_single_netcdf_p3_new(loc, pattern = None, vars_to_calc=['VEL', 'DBZ', 'AVEL', 'ADBZ', 'SVEL', 'ISO', 'PGG', 'RG', 'NRG']):
    # The first variable is the original (pre-clean) key
    # The "result' variabe is the cleaned up field.
    # To create Y, we compare the "key" to the "cleaned" variable.

    # ALT and all the AVGs/SDs are added by the script.
    # Note that once we compute Y, we remove the cleaned up field
    #     because it isn't needed anymore for training or testing.


    my_vars = [ 'VEL', 'DBZ', 'SQI']
    my_avgs = [ 'AVEL', 'ADBZ', 'ASQI' ]
    my_stds = [ 'SVEL', 'SDBZ', 'SSQI' ]
    my_iso = ['ISO']
    my_PGG = ['PGG']
    my_RG = ['RG']
    my_NRG = ['NRG']


    ISO_VAR = 'DBZ' 

    alt_index = len(my_vars)        # Index of altitude
    avg_offset = alt_index + 1      # Averages will be after the altitude
    std_offset = avg_offset + len(my_avgs)
    iso_index = std_offset + len(my_stds)
    PGG_index = iso_index + 1
    RG_index = PGG_index + 1
    NRG_index = RG_index + 1



    #ELEV_index = NRG_index + 1


    file_list = expand_path(loc, pattern)
    print(file_list)
    num_files = len(file_list)

    max_x, max_y = get_dim(file_list)
    flat_size = max_x * max_y
    num_cols = num_files * flat_size
    dim = (len(vars_to_calc), num_cols)

    X = np.empty(dim)
    ob_index = 0
    fill_val = None

    ###ISAAC MODIFICATIONS 

    for path in file_list:

        ALT_COMPLETE = False 

        PGG_matrix = [[]]
        ALT_matrix = [[]]

        print("OK")
        print ('Reading ', path)
        nc_ds = nc4.Dataset(path, 'r')

        # read the variables we need to compute altitude
        max_time = nc_ds.dimensions['time'].size
        max_range = nc_ds.dimensions['range'].size
        rnge = nc_ds.variables['range']
        plane_alts = nc_ds.variables['altitude'] # (time)
        ray_angles = nc_ds.variables['elevation'] # (time)
        ranges = nc_ds.variables['range'] # (time)
        azimuths = nc_ds.variables['azimuth'] # (time)


        ###Isaac modifications 
        ###This is clunky as heck but just wanted to get something working 
        for i, var in enumerate(vars_to_calc): 

            if var in my_vars: 

                currmoment = nc_ds.variables[var]
                fill_val = getattr(currmoment, '_FillValue')

                # Flatten it and append to the X array
                one_d = (currmoment[:]).reshape(-1)
                X[i, range(ob_index, ob_index + one_d.size)] = one_d

            elif var in my_avgs:

                print('Computing averages for ', var)
                var_avrg = compute_avgs(nc_ds.variables[var[1:]], fill_val)
                one_d = (var_avrg[:]).reshape(-1)
                X[i, range(ob_index, ob_index + one_d.size)] = one_d

            elif var in my_stds:

                print('Computing stds for ', var)
                var_std = compute_std(nc_ds.variables[var[1:]], fill_val)
                one_d = (var_std[:]).reshape(-1)
                X[i, range(ob_index, ob_index + one_d.size)] = one_d 
            
            elif var == 'ISO': 

                #This section calculates the isolation of radar gates
                print('Computing isolation of ', var)
                var_iso = compute_isolation(nc_ds.variables[ISO_VAR], fill_val)
                one_d_iso = (var_iso[:]).reshape(-1)
                X[i, range(ob_index, ob_index + one_d.size)] = one_d_iso

            elif var == 'PGG' or var == 'ALT': 
                
                if ALT_COMPLETE: 
                    if var == 'PGG': 
                        one_d_pgg = (PGG_matrix[:]).reshape(-1)
                        X[i, range(ob_index, ob_index + one_d_pgg.size)] = one_d_pgg
                    else: 
                        one_d_alt = (ALT_matrix[:]).reshape(-1)
                        X[i, range(ob_index, ob_index + one_d_alt.size)] = one_d_alt 

                    continue 
                else: 
                    heights = np.empty( (max_time, max_range) )
                    probgg = np.empty( (max_time, max_range) )
                
                    for time_idx in range(max_time):
                        for range_idx in range(max_range):
                            tangle = float(ray_angles[time_idx].data)
                            trange = float(ranges[range_idx].data)
                            talt   = float(plane_alts[time_idx].data)
                            taz    = float(azimuths[time_idx].data)

                            heights[time_idx, range_idx] = computeHtKmAirborn(
                                tangle, trange, talt)

                            probgg[time_idx, range_idx] = probGroundGates(
                                tangle, trange, talt, taz, max_range)

                    ALT_matrix = heights
                    PGG_matrix = probgg
                    if var == 'PGG': 
                        one_d = (PGG_matrix[:]).reshape(-1)
                        X[i, range(ob_index, ob_index + one_d.size)] = one_d
                    else:   
                        one_d = (ALT_matrix[:]).reshape(-1) 
                        X[i, range(ob_index, ob_index + one_d.size)] = one_d

            elif var == 'RG': 
                rg = np.empty( (max_time, max_range) )

                for time_idx in range(max_time):
                    for range_idx in range(max_range):
                        rg[time_idx, range_idx] = rnge[range_idx]
                        #elev[time_idx, range_idx] = rnge[time_idx]
                        ralt = float(plane_alts[time_idx].data)

                one_d = (rg[:]).reshape(-1)
                X[i, range(ob_index, ob_index + one_d.size)] = one_d

            elif var == 'NRG': 

                nrg = np.empty( (max_time, max_range) )
                #elevat = np.empty( (max_time, max_range) )
                for time_idx in range(max_time):
                    for range_idx in range(max_range):
                        rg[time_idx, range_idx] = rnge[range_idx]
                        #elev[time_idx, range_idx] = rnge[time_idx]
                        ralt = float(plane_alts[time_idx].data)
                        nrg[time_idx, range_idx] = (rnge[range_idx])/ralt

                one_d = (nrg[:]).reshape(-1)
                X[i, range(ob_index, ob_index + one_d.size)] = one_d
                # Read the "feature" variables


            else: 
                raise Exception("UNKNOWN VARIABLE NAME" + str(var)) 
            

        nc_ds.close()
        #The line below is vital to continued operation with multiple files
        ob_index += one_d.size
        
        # for var in range(len(my_vars)):
        #     my_var = nc_ds.variables[my_vars[var]]
        #     fill_val = getattr(my_var, '_FillValue')
        #     print(np.shape(my_var))

        #     # Flatten it and append to the X array
        #     one_d = (my_var[:]).reshape(-1)
        #     X[var, range(ob_index, ob_index + one_d.size)] = one_d

        #     # Compute the neighbor average, and append it to X

        #     if var >= 3:        # Don't average VG since we will remove it
        #         continue

        #     print('Computing averages for ', my_vars[var])
        #     var_avrg = compute_avgs(my_var, fill_val)
        #     one_d = (var_avrg[:]).reshape(-1)
        #     X[var + avg_offset, range(ob_index, ob_index + one_d.size)] = one_d

        #     print('Computing std deviations for ', my_vars[var])
        #     var_std = compute_std(my_var, fill_val)
        #     one_d = (var_std[:]).reshape(-1)
        #     X[var + std_offset, range(ob_index, ob_index + one_d.size)] = one_d

        # #This section calculates the isolation of radar gates
        # print('Computing isolation of ', my_vars[1])
        # var_iso = compute_isolation(nc_ds.variables[my_vars[1]], fill_val)
        # one_d_iso = (var_iso[:]).reshape(-1)
        # X[iso_index, range(ob_index, ob_index + one_d.size)] = one_d_iso

        # # Compute and add the height
        
        # print('Computing altitudes and probability of ground gates for ', max_time * max_range, ' entries')

        # #This is where altitude is computed and how probability of ground gates should be implemented
        # heights = np.empty( (max_time, max_range) )
        # probgg = np.empty( (max_time, max_range) )
    
        # for time_idx in range(max_time):
        #      for range_idx in range(max_range):
        #          tangle = float(ray_angles[time_idx].data)
        #          trange = float(ranges[range_idx].data)
        #          talt   = float(plane_alts[time_idx].data)
        #          taz    = float(azimuths[time_idx].data)

        #          heights[time_idx, range_idx] = computeHtKmAirborn(
        #             tangle, trange, talt)

        #          probgg[time_idx, range_idx] = probGroundGates(
        #             tangle, trange, talt, taz, max_range)

        # one_d = (heights[:]).reshape(-1)
        # X[alt_index, range(ob_index, ob_index + one_d.size)] = one_d

        # one_d = (probgg[:]).reshape(-1)
        # X[PGG_index, range(ob_index, ob_index + one_d.size)] = one_d

        # print('Computing Range and Normalized Range')
        # rg = np.empty( (max_time, max_range) )
        # nrg = np.empty( (max_time, max_range) )
        # #elevat = np.empty( (max_time, max_range) )
        # for time_idx in range(max_time):
        #      for range_idx in range(max_range):
        #          rg[time_idx, range_idx] = rnge[range_idx]
        #          #elev[time_idx, range_idx] = rnge[time_idx]
        #          ralt = float(plane_alts[time_idx].data)
        #          nrg[time_idx, range_idx] = (rnge[range_idx])/ralt

        # one_d = (rg[:]).reshape(-1)
        # X[RG_index, range(ob_index, ob_index + one_d.size)] = one_d

        # one_d = (nrg[:]).reshape(-1)
        # X[NRG_index, range(ob_index, ob_index + one_d.size)] = one_d
        
        # #one_d = (elevat[:]).reshape(-1)
        # #X[ELEV_index, range(ob_index, ob_index + one_d.size)] = one_d
        
        # nc_ds.close()
        # #The line below is vital to continued operation with multiple files
        # ob_index += one_d.size

    print(np.shape(X))
    return X

In [13]:
DATA_PATH = [str("/Users/ischluesche/Documents/Grad_School/Research/RadarQC/NOAA/TESTING/")]
#CASE_PATHS = [DATA_PATH + "/BAMEX/", DATA_PATH + "/HAGUPIT/", 
#              DATA_PATH + "/RITA/", DATA_PATH + "/VORTEX/"]

for path in DATA_PATH:
    curr_files = os.listdir(path)
    for file in curr_files: 
        print(path + file)
        %timeit load_single_netcdf_p3_new(path + file)

/Users/ischluesche/Documents/Grad_School/Research/RadarQC/NOAA/TESTING/cfrad.20220918_125122.944_to_20220918_125126.918_N42RF-TS_AIR.nc
['/Users/ischluesche/Documents/Grad_School/Research/RadarQC/NOAA/TESTING/cfrad.20220918_125122.944_to_20220918_125126.918_N42RF-TS_AIR.nc']
OK
Reading  /Users/ischluesche/Documents/Grad_School/Research/RadarQC/NOAA/TESTING/cfrad.20220918_125122.944_to_20220918_125126.918_N42RF-TS_AIR.nc
Computing averages for  AVEL
Computing averages for  ADBZ
Computing stds for  SVEL
Computing isolation of  ISO


UnboundLocalError: local variable 'nrg' referenced before assignment

## Load the model

In [7]:
## For now this uses pickle, but I need to migrate to a new method, feel free to replace with your own

#Load the rf Model that will process all the data - this is where you change the model you want to use
rf_pkl_filename = './rf_model/rf_21_14_firstcut_NOAA_P3.pkl'

#This part of the model loading code stays constant
load_rf_model_pkl = open(rf_pkl_filename, 'rb')
stored_rf_model = pickle.load(load_rf_model_pkl)

In [7]:
###Grab calculated paramters and output it to an h5 files 

file_array = ["./cfrad/cfrad.20220918_124128.197_to_20220918_124132.176_N42RF-TS_AIR.nc"]
X = load_single_netcdf_p3_new(file_array[0]) 


ALT INDEX3
ISO_INDEX: 10
PGG_INDEX: 11
RG_INDEX: 12
Reading  ./cfrad/cfrad.20220918_124128.197_to_20220918_124132.176_N42RF-TS_AIR.nc
(360, 721)
Computing averages for  VEL
Computing std deviations for  VEL
(360, 721)
Computing averages for  DBZ
Computing std deviations for  DBZ
(360, 721)
Computing averages for  SQI
Computing std deviations for  SQI
Computing isolation of  DBZ
Computing altitudes and probability of ground gates for  259560  entries
Computing Range and Normalized Range
(15, 259560)


In [8]:
header = [ 'VEL', 'DBZ', 'SQI', 'ALT', 'AVEL', 'ADBZ', 'ASQI', 'SVEL', 'SDBZ', 'SSQI', 'ISO', 'PGG', 'RG', 'NRG', 'EMPTY']


In [33]:
cfrad["VEL"][:,:]

masked_array(
  data=[[ 0.  ,  0.  ,  0.03, ...,  0.  ,  0.  ,  0.  ],
        [ 0.  ,  0.  ,  0.04, ...,  0.  ,  0.  ,  0.  ],
        [ 0.  , -0.01, -0.06, ...,  0.  ,  0.  ,  0.  ],
        ...,
        [ 0.  ,  0.  ,  0.03, ...,  0.  ,  0.  ,  0.  ],
        [ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ],
        [ 0.  ,  0.01,  0.06, ...,  0.  ,  0.  ,  0.  ]],
  mask=False,
  fill_value=1e+20)

In [51]:
compute_avgs(cfrad["VEL"][0:5,0:5], fill_val=-3222) * 100

array([[ 8.67361738e-17,  1.07272727e+01,  3.45714286e+01,
         3.61818182e+01,  4.72500000e+01],
       [ 3.63636364e-01,  1.20666667e+01,  3.88421053e+01,
         4.64666667e+01,  5.08181818e+01],
       [ 4.28571429e-01,  1.20000000e+01,  3.71666667e+01,
         4.74736842e+01,  5.80000000e+01],
       [ 2.72727273e-01,  9.00000000e+00,  3.57894737e+01,
         4.16666667e+01,  4.48181818e+01],
       [-1.25000000e-01,  7.81818182e+00,  3.20714286e+01,
         3.70000000e+01,  4.42500000e+01]])

In [47]:
np.nanmean(cfrad["VEL"][0:4, 0:4])

0.113125

In [43]:
toy_arr = cfrad["VEL"][0:4, 0:4]

In [27]:
X[4, :].reshape(721,360)

array([[ 8.67361738e-19,  1.07272727e-01,  3.45714286e-01, ...,
        -2.40714286e-01, -3.90000000e-01, -5.88571429e-01],
       [-9.05714286e-01, -8.83571429e-01, -7.00714286e-01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  3.63636364e-03,  1.20666667e-01, ...,
         6.30526316e-01,  2.74210526e-01,  2.10526316e-02],
       ...,
       [-1.21773684e+01, -1.08489474e+01, -1.16205263e+01, ...,
         0.00000000e+00,  0.00000000e+00,  1.25000000e-02],
       [ 2.17272727e-01,  6.23571429e-01,  1.07500000e+00, ...,
        -1.30635714e+01, -1.19614286e+01, -1.32921429e+01],
       [-1.47264286e+01, -1.08992857e+01, -9.37714286e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [11]:
utils.compute_avg

NameError: name 'utils' is not defined

In [9]:
currset = nc4.Dataset(file_array[0], 'r+')
timedim = currset.dimensions['time']
rangedim = currset.dimensions['range']

for i, variable in enumerate(header): 

    try: 
        currset.createVariable(variable, 'f8', ('time', 'range'))
    ##If already defined, just overwrite 
    except RuntimeError as e:
        currset[variable][:, :] = X[i].reshape([len(timedim), len(rangedim)])

    
    currset[variable].long_name = 'New ML edited VG field'
    currset[variable].units = 'm/s'
    currset[variable].add_offset = -0.0
    #currset[variable].scale_factor = 0.01
    currset[variable].grid_mapping = 'grid_mapping'
    currset[variable].sampling_ratio = 1.0
    currset[variable].coordinates = 'time range'

currset.close() 
    


OSError: [Errno -101] NetCDF: HDF error: b'./cfrad/cfrad.20220918_124128.197_to_20220918_124132.176_N42RF-TS_AIR.nc'

In [39]:
nc4.Dataset(file_array[0], 'r+').set_fill_off()

In [40]:
nc4.Dataset(file_array[0], 'r+')

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.7
    Sub_conventions: CF-Radial instrument_parameters radar_parameters radar_calibration platform_velocity geometry_correction
    version: CF-Radial-1.4
    title: RAW-ALL
    institution: 
    references: Conversi
    source: Sigmet I
    history: !   time-now 11/03/2023 17:19:20.000 GMT
    sweep-directory "./"
    radar-names N42RF-TS
    first-sweep first
    last-sweep last
    version last
no-new-version
optimal-beamwidth is 2.0 degrees
a-speckle is 3 gates
! for-each-ray
copy DBZ to DT
copy VEL to VG
clear-bad-flags
copy-bad-flags from DT
assert-bad-flags in VG
clear-bad-flags
remove-aircraft-motion in VG
remove-ring in VG from 0.0 to 0.275 km.
remove-ring in VG from 51.8 to 55.0 km.
threshold VG on SQI below 0.3
remove-only-surface in VG
copy-bad-flags from VG
assert-bad-flags in DT
copy DT to SLA
assign-value 9.0 to SLA
copy-bad-flags from DT
assert-bad-flags in SLA
re

In [27]:
currset = nc4.Dataset(file_array[0], 'r+')
range_dim = currset["range"].size
time_dim = currset["time"].size

In [26]:
currset.crea

721

## Run the QC: Should ttake ~ 5 mins

In [12]:
#An iterator to loop through all files in the pass - for now it is just 1
file_array = glob.glob('./cfrad/cfrad.*')
#This loop will pull fields from the cfradial, clean them with the ml model, and write them back into the file
for j in file_array:
    print(j)
    #Predictors from the radar file are fed in to allow the model to work
    X = load_single_netcdf_p3_new(j)

    # Create a copies of relevant fields to write into the file
    ZE = np.copy(X[1])
    VGE = np.copy(X[0])
    ALT = np.copy(X[3])
    PGG = np.copy(X[11])
    
    #Because the model was not trained on null data, this tells you where predictions need to be autoassigned
    #Doing this before the X array gets trimmed
    NCP_idx = np.argwhere(X[2,:] <= 0.2)
    NCP_idx = np.squeeze(NCP_idx)
    PGG_idx = np.argwhere(X[11,:] >= 1)
    PGG_idx = np.squeeze(PGG_idx)

    #Cut out predictors that were eliminated by Lasso Logistic Regression - MOVED HERE SO CORRECT FIELD IS COPIED
    good_idx = np.array([1,2,3,4,11,13])
    X = np.array(X)[good_idx.astype(int),:]
    print(np.shape(X))

    #This is where the model makes predictions on your radar data
    print("PREDICTING USING X OF SHAPE " + str(np.shape(X)))
    preds = stored_rf_model.predict(X.T).reshape(-1)
    print(len(np.argwhere(preds==1)))
    #This sets a fill value that will be assigned to bad data when being written back into cfradial format
    fill_val = int(-32768)
    fill_val = np.float64(fill_val)
    fvid = np.argwhere(ZE == fill_val)
    fvid = np.squeeze(fvid)
    #Missing data is automatically given a bad prediction
    print(len(fvid))
    print(len(NCP_idx))
    print(len(PGG_idx))
    for i in fvid:
        preds[i] = 0
    for i in NCP_idx:
        preds[i] = 0
    for i in PGG_idx:
        preds[i] = 0
    #These are the indices where the model has predicted bad data
    print(len(np.argwhere(preds==1)))
    idx = np.argwhere(preds == 0)
    print(len(idx))
    #At each instance of bad data predicition, the fill_val is assigned
    ZE[idx] = fill_val
    VGE[idx] = fill_val
    #This step sets the bad data, marked by fill value, to missing using a masked array
    ZE = np.ma.masked_equal(ZE, fill_val)
    VGE = np.ma.masked_equal(VGE, fill_val)
    PGG = np.ma.masked_equal(PGG, fill_val)
    #OPEN THE CFRADIAL FILE
    nc_ds = nc4.Dataset(j, 'r+')
    timedim = nc_ds.dimensions['time']
    rangedim = nc_ds.dimensions['range']
    #This step reshapes the edited array into the proper format for the cfradial file
    ZE = ZE.reshape([len(timedim), len(rangedim)])
    VGE = VGE.reshape([len(timedim), len(rangedim)])
    ALT = ALT.reshape([len(timedim), len(rangedim)])
    PGG = PGG.reshape([len(timedim), len(rangedim)])
    #At this step the new fields are created in the cfradial file to become the new ML edited versions of the data
    ze = nc_ds.createVariable('ZE1', 'i2', ('time','range',), fill_value= fill_val)
    ze.long_name = 'New ML edited DBZ field'
    ze.units = 'dBz'
    ze.add_offset = -0.0
    ze.scale_factor = 0.01
    ze.grid_mapping = 'grid_mapping'
    ze.sampling_ratio = 1.0
    ze.coordinates = 'time range'
    ze[:,:] = ZE

    vge = nc_ds.createVariable('VGE1', 'i2', ('time','range',), fill_value= fill_val)
    vge.long_name = 'New ML edited VG field'
    vge.units = 'm/s'
    vge.add_offset = -0.0
    vge.scale_factor = 0.01
    vge.grid_mapping = 'grid_mapping'
    vge.sampling_ratio = 1.0
    vge.coordinates = 'time range'
    vge[:,:] = VGE
    
    alt = nc_ds.createVariable('ALT1', 'i2', ('time','range',), fill_value= fill_val)
    alt.long_name = 'Alt field'
    alt.units = 'm/s'
    alt.add_offset = -0.0
    alt.scale_factor = 0.01
    alt.grid_mapping = 'grid_mapping'
    alt.sampling_ratio = 1.0
    alt.coordinates = 'time range'
    alt[:,:] = ALT
    
    pgg = nc_ds.createVariable('PGG1', 'i2', ('time','range',), fill_value= fill_val)
    pgg.long_name = 'New PGG field'
    pgg.units = 'unitless'
    pgg.add_offset = -0.0
    pgg.scale_factor = 0.01
    pgg.grid_mapping = 'grid_mapping'
    pgg.sampling_ratio = 1.0
    pgg.coordinates = 'time range'
    pgg[:,:] = PGG
    
    #The cfradial file must be closed to complete the process
    nc_ds.close()
print('done')

./cfrad/cfrad.20220918_124128.197_to_20220918_124132.176_N42RF-TS_AIR.nc
ALT INDEX3
ISO_INDEX: 10
PGG_INDEX: 11
RG_INDEX: 12
Reading  ./cfrad/cfrad.20220918_124128.197_to_20220918_124132.176_N42RF-TS_AIR.nc
(360, 721)
Computing averages for  VEL
Computing std deviations for  VEL
(360, 721)
Computing averages for  DBZ
Computing std deviations for  DBZ
(360, 721)
Computing averages for  SQI
Computing std deviations for  SQI
Computing isolation of  DBZ
Computing altitudes and probability of ground gates for  259560  entries
Computing Range and Normalized Range
(15, 259560)
(6, 259560)
PREDICTING USING X OF SHAPE (6, 259560)
57572
144877
80067
110500
57534
202026


RuntimeError: NetCDF: String match to name in use

## Examine the Output

### I don't have PyArt installed locally, so I left an image of the QCed Output in the Folder using HawkEdit

### If you have PyArt installed, try working with these plotting commands

In [ ]:
# Plot the Raw and QCed Velocity
import matplotlib.pyplot as plt
import pyart
#from pyart.testing import get_test_data
path = './cfrad/cfrad.20220918_124128.197_to_20220918_124132.176_N42RF-TS_AIR.nc'
radar = pyart.io.read(path)
radar.scan_type = 'rhi' 

# Setup the display
display = pyart.graph.AirborneRadarDisplay(radar)
fig = plt.figure(figsize = (7,8), dpi = 350)
ax1 = fig.add_subplot(211)
c1 = display.plot('VEL', cmap = "RdYlBu", title = "Raw Velocity - Earl Case in Mainly Clear Air - ~ 13:25 UTC 9/7/22", vmin = -15, vmax = 15, colorbar_flag = False, axislabels = ((''),('Distance above radar (km)')))
display.plot_range_rings([10, 20, 30, 40, 50])
display.set_limits(ylim = [-10,19], xlim = [-55,55])

ax2 = fig.add_subplot(212)
display.plot('VGE1', cmap = "RdYlBu", title = "Machine Learning QC Velocity", vmin = -15, vmax = 15, colorbar_flag = False, axislabels = ((''),('Distance above radar (km)')))
display.plot_range_rings([10, 20, 30, 40, 50])
display.set_limits(ylim = [-10,19], xlim = [-55,55])

cbar_ax = fig.add_axes([0.915,.15,0.025,.7])

display.plot_colorbar(cax = cbar_ax, ax = cbar_ax, mappable = c1, field='VEL', label="P3 TDR Doppler Velocity (m s$^{-1}$)", orient='vertical')

In [ ]:
# Plot the Altitude and PGG Predictors
import matplotlib.pyplot as plt
import pyart
#from pyart.testing import get_test_data
path = './cfrad/cfrad.20220918_124128.197_to_20220918_124132.176_N42RF-TS_AIR.nc'
# Locate the test data and read in using main read method
#filename = get_test_data(path)
radar = pyart.io.read(path)
radar.scan_type = 'rhi' 

# Setup the display
display = pyart.graph.AirborneRadarDisplay(radar)
fig = plt.figure(figsize = (7,8), dpi = 350)
ax1 = fig.add_subplot(211)
c1 = display.plot('ALT4', cmap = "Spectral_r", title = "Altitude Field - Colorbar Range from -10 to 25 km - Bad Elevation Field", vmin = -10, vmax = 25, colorbar_flag = False, axislabels = ((''),('Distance above radar (km)')))
display.plot_range_rings([10, 20, 30, 40, 50])
display.set_limits(ylim = [-10,19], xlim = [-55,55])

ax2 = fig.add_subplot(212)
display.plot('PGG4', cmap = "viridis", title = "PGG Field - Colorbar Range from 0 to 1", vmin = 0, vmax = 1, colorbar_flag = False, axislabels = ((''),('Distance above radar (km)')))
display.plot_range_rings([10, 20, 30, 40, 50])
display.set_limits(ylim = [-10,19], xlim = [-55,55])

# cbar_ax = fig.add_axes([0.915,.15,0.025,.7])

# display.plot_colorbar(cax = cbar_ax, ax = cbar_ax, mappable = c1, field='VEL', label="P3 TDR Doppler Velocity (m s$^{-1}$)", orient='vertical')